In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of PFNExperiments.LinearRegression.ComparisonModels.SGLD failed: Traceback (most recent call last):
  File "c:\Users\arik_\Documents\Dokumente\Job_Clausthal\PFNs\Repository\PFNExperiments\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\arik_\Documents\Dokumente\Job_Clausthal\PFNs\Repository\PFNExperiments\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\arik_\Documents\Dokumente\Job_Clausthal\PFNs\Repository\PFNExperiments\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\arik_\Documents\Dokumente\Job_Clausthal\PFNs\Repository\PFNExperiments\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\arik_

In [18]:
#!pip install langevin-sampling

In [19]:
import torch
import pyro.poutine as poutine
import pyro

from pyro.infer.autoguide import AutoDelta
from pyro.infer import SVI, Trace_ELBO
import torch
from pyro.optim import PyroOptim

from langevin_sampling.SGLD import SGLD as SGLD_org

In [20]:
from PFNExperiments.LinearRegression.ComparisonModels.SGLD import SGLD
from PFNExperiments.LinearRegression.GenerativeModels.GenerateDataLM_Examples import make_lm_program_ig
from PFNExperiments.LinearRegression.GenerativeModels.LM_abstract import return_only_y, print_code

In [21]:
from torch.optim import Adam

In [22]:
pprogram = make_lm_program_ig(
)

pprogram_return_only_y = return_only_y(pprogram)

In [23]:
X = torch.randn(100, 10)
r = pprogram(X)
y = r['y']

In [24]:
guide = AutoDelta(pprogram_return_only_y)

Trace_ELBO().differentiable_loss(pprogram_return_only_y, guide, X, y)

optim = PyroOptim(SGLD_org, {"lr": 1e-3})

svi = SVI(pprogram_return_only_y, guide, optim, loss=Trace_ELBO())

svi = SVI(pprogram_return_only_y, guide, optim, loss=Trace_ELBO())

In [25]:
loss = svi.step(X, y)

print(list(guide.parameters()))
print(guide.median())
print(loss)

[Parameter containing:
tensor(2.6529, requires_grad=True), Parameter containing:
tensor([ 0.0574, -0.9531, -0.0731, -0.0808,  0.1744,  0.8579,  0.8908, -0.0966,
         0.2126, -0.4793], requires_grad=True)]
{'sigma_squared': tensor(14.1956), 'beta': tensor([ 0.0574, -0.9531, -0.0731, -0.0808,  0.1744,  0.8579,  0.8908, -0.0966,
         0.2126, -0.4793])}
395.089075088501


In [26]:
sampler = SGLD(
    pprogram=pprogram_return_only_y,
)

In [27]:
from PFNExperiments.LinearRegression.ComparisonModels.SGLD import SGLD

In [28]:
sampler = SGLD(
    pprogram =  pprogram_return_only_y,
    n_samples = 10,
    n_warmup = 10,
    shuffle_samples = False,
    optim_kwargs = {"lr": 1e-3}
)

In [29]:
r2 = sampler.sample_posterior(X, y)

In [30]:
r2

{'sigma_squared': tensor([3.8518, 3.7499, 3.7993, 3.4612, 3.3583, 3.2942, 3.4017, 3.1500, 3.2905,
         3.0112]),
 'beta': tensor([[ 0.1108, -0.8856,  0.0665, -0.1134,  0.2234,  0.8938,  1.0622, -0.1907,
           0.4166, -0.0596],
         [ 0.0728, -0.8529,  0.0673, -0.1048,  0.2596,  0.9013,  1.0804, -0.1925,
           0.3810,  0.0351],
         [ 0.0717, -0.8827,  0.0547, -0.1081,  0.2419,  0.8750,  1.0946, -0.2483,
           0.3607,  0.1182],
         [ 0.0552, -0.8112,  0.1722, -0.0428,  0.2083,  0.8350,  1.0851, -0.2308,
           0.4320,  0.1329],
         [ 0.0473, -0.8402,  0.2845, -0.0238,  0.2617,  0.8186,  1.1084, -0.1289,
           0.4720,  0.1682],
         [ 0.0164, -0.8349,  0.3697, -0.0310,  0.2589,  0.7470,  1.0804, -0.1501,
           0.4254,  0.1918],
         [ 0.0378, -0.8320,  0.2987, -0.0265,  0.2854,  0.7196,  1.0501, -0.0783,
           0.4341,  0.2884],
         [ 0.0405, -0.8238,  0.2352, -0.1410,  0.3098,  0.6746,  1.0779, -0.0806,
           0.429